In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from functools import partial
import nltk
from src.contextual_bart import ContextualisedBartModel,BartForContextualRecovery,SimplifiedBeamSearch
from src.dataset_processor import load_all_data
from src.utils import SmartCollator, get_args, setuptokenizer
from src.dataset_processor import (
    ContextGenerationDataset,
)
from transformers import BartTokenizer, BartConfig,BartForConditionalGeneration
from src.model_utils import CustomTrainer, get_training_arguments
import torch
from src.config import DATASET_PATH
from transformers.trainer_callback import EarlyStoppingCallback
import pickle as pk
import torch
from transformers import (    AutoTokenizer,
          AutoModelForSeq2SeqLM,
         LogitsProcessorList,    MinLengthLogitsProcessor, StoppingCriteriaList, MaxLengthCriteria,
         TopKLogitsWarper, TemperatureLogitsWarper,BeamSearchScorer,)

nltk.download("punkt")

DATASET_PATH = "summarisation_data2/"

def generate_data():

    # load the dataset

    train_data_packet = load_all_data(DATASET_PATH, mode="train")
    dev_data_packet = load_all_data(DATASET_PATH, mode="dev")
    test_data_packet = load_all_data(DATASET_PATH, mode="test")

    print(f"Training Data-size: {len(train_data_packet)}")
    print(f"Test Data-size: {len(test_data_packet)}")
    return train_data_packet,dev_data_packet,test_data_packet

[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
train_data_packet,dev_data_packet,test_data_packet = generate_data()

processing files:  ['summarisation_data2/xsum_train.csv']


ValueError: too many values to unpack (expected 3)

In [48]:
train_data_packet[0]

ContextualGenerationData(input='Walls, hoardings, shop shutters and camper vans will be painted live by artists from South America and Europe throughout the weekend. Upfest, which began in 2008, is centred on Bedminster\'s North Street. Some murals will remain until the next Upfest, including huge paintings on the Tobacco Factory venue and the Red Point climbing centre. Visitors to the festival were able to watch 275 artists from 25 countries painting during live shows over the weekend. Among those taking part are Inkie, My Dog Sighs, Gamma, Lonac and Dutch due Telmo Miel. As well as walls and vehicles, Upfest has its own 14m (45ft) long version of a New York-style subway train, created in 2012, for artists to decorate. At the Tobacco Factory Argentinian artist Martin Ron - known for huge surreal paintings in Buenos Aires - was painting a 2,025 sq ft (188 sq m) mural of his girlfriend. Bristol, the birthplace of Banksy, has an international reputation for its street art. Gallery owner 

In [3]:
import copy
from src.dataset_processor import ContextualGenerationData
import numpy as np

In [6]:
import copy
import random
from dataclasses import asdict
def boundarySplit(data: ContextualGenerationData,
                  boundary_limits = (0.25, 0.8),
                  max_context_tokens=[50, 200],
                  nb_examples= 3):
    data_pack=[]
    max_context = np.arange(max_context_tokens[0], max_context_tokens[-1], step=5)
    random.shuffle(max_context)
    for idx  in range(nb_examples):
        boundary_portion = np.round(np.random.uniform(
                    size=(1,),
                    low=boundary_limits[0],
                    high=boundary_limits[1],
                )[0],2)
        d = copy.deepcopy(data)
        d.boundary_proportion = (boundary_portion,int(max_context[idx]))
        data_pack.append(asdict(d))
    return data_pack

In [55]:
new_train_data = [boundarySplit(data) for data in train_data_packet]
new_test_data =  [boundarySplit(data) for data in test_data_packet]
new_validation_data = [boundarySplit(data) for data in dev_data_packet]

In [4]:
import itertools
import pandas as pd

In [56]:



def getdataframe(pack):
    data_pack = pd.DataFrame(pack)
    #data_pack= data_pack.drop_duplicates(subset = ["output"],keep="last")
    return data_pack
new_train_data = getdataframe(list(itertools.chain(*new_train_data)))
new_test_data = getdataframe(list(itertools.chain(*new_test_data)))
new_validation_data = getdataframe(list(itertools.chain(*new_validation_data)))

In [59]:
len(train_data_packet),len(new_train_data)

(203083, 609249)

In [39]:
import os
os.makedirs("summarisation_data2/")

In [60]:
new_train_data.to_csv('summarisation_data2/xsum_train.csv')
new_test_data.to_csv('summarisation_data2/xsum_dev.csv')
new_validation_data.to_csv('summarisation_data2/xsum_test.csv')

In [9]:
from dataclasses import asdict

In [5]:
def read_csv2(file_name):
    data = pd.read_csv(file_name).values
    pack=[]
    for idx,document,summary,_,_,boundary in data:
        if len(str(document).split()) > 2:
            pack.append(ContextualGenerationData(input=str(document),
                                                 boundary_proportion=boundary,output=str(summary)))
    return pack

In [6]:
gg= read_csv2('summarisation_data2/xsum_dev.csv')

In [68]:
type(eval(gg[0].boundary_proportion))

tuple

In [7]:
tokenizer = setuptokenizer(
        model_base="facebook/bart-base", special_tokens=[], additional_tokens=['[SEP]']
    )

In [8]:
zz= gg[-1]

In [9]:
passage_pack = tokenizer(
            zz.input,
            add_special_tokens=False,
            return_tensors="pt",
        )

passage_seq = passage_pack["input_ids"].flatten()
passage_attention = passage_pack["attention_mask"].flatten()

In [21]:
passage_seq.shape

torch.Size([147])

In [10]:
start_position = round(passage_seq.shape[0]*eval(zz.boundary_proportion)[0])
end_position = start_position+ eval(zz.boundary_proportion)[-1]

In [11]:
start_position,end_position,passage_seq.shape[0]

(283, 338, 545)

In [36]:
zz.boundary_proportion

(0.34, 200)

In [37]:
passage_seq[start_position:end_position]

tensor([15268,   583, 14961,    23,   601,    35,   541,  5050,    15,   307,
            4,  1868,   604,    58,   551,     7,  1098,     6,    19,     5,
         7801,  1001,   225,  1393,    26,     7,    28,  2008,     4,   522,
           26,    10,  2182,  4408,    11,     5, 22264,   962,    11,  1098,
           15,   273,     4,   264,    56,    57,   551,     7,  3929,  4690,
         2392,     6,    11,  8353,   111,   147,     5,  7801,  1001,   225,
         1393,    16,   145,  3032,   111,    71,     5,  3213,     6,    95,
           81,     5,  1424,    31,     5,  1398,  1891,  9959,  1139,     9,
         1745,  1054,     4,    20,  1370,    26,    69,   220,     9, 24438,
            8,     5, 16489,    33,    57,  3978,     4])

In [12]:
from dataclasses import dataclass
from typing import List, Optional, Tuple
from src.model_utils import Features
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import logging
logger = logging.getLogger(__name__)

@dataclass
class ContextWithBoundaryFeatures:
    input_ids: List[int]
    attention_mask: List[int]
    labels: Optional[List[int]]
    decoder_attention_mask: Optional[List[int]]
    boundary: Optional[Tuple[int,int]]


In [13]:
model_base = "facebook/bart-base"
tokenizer = setuptokenizer(
        model_base=model_base, special_tokens=[], additional_tokens=["[SEP]"]
    )
    # tokenizer.add_tokens([])
max_seq_len= 512
is_not_auto_encoder_data=True
train_dataset = ContextGenerationDatasetBoundary(
        tokenizer=tokenizer,
        nb_records=len(train_data_packet),
        max_len=max_seq_len,
        context_seperator="[SEP]",
        is_auto_encoder_data=not is_not_auto_encoder_data,
        use_special_token=True,
    )
train_dataset.change_data_mode(1)
train_dataset.set_record(train_data_packet)

NameError: name 'ContextGenerationDatasetBoundary' is not defined

In [ ]:
train_dataset[0]

In [14]:
import logging
import math
import random
from dataclasses import dataclass
from logging import Logger
from typing import List, Optional, Tuple, Union
import numpy as np
import torch
import torch.nn as nn
from transformers import (
    BeamSearchScorer,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    TemperatureLogitsWarper,
    TopKLogitsWarper,
)
from transformers.models.bart.modeling_bart import (
    BartConfig,
    BartDecoder,
    BartEncoderLayer,
    BartLearnedPositionalEmbedding,
    BartPretrainedModel,
    BaseModelOutput,
    CrossEntropyLoss,
    Seq2SeqLMOutput,
    Seq2SeqModelOutput,
    _expand_mask,
    shift_tokens_right,
)
from src.model_utils import EncoderOutputs

import logging
logger = logging.getLogger(__name__)



In [20]:
yuy = torch.LongTensor([(23,100),(56,198)])

In [24]:
a,b=yuy[0]

In [15]:
from src.contextual_bart import BartEncoderBoundary

In [16]:
encoder = BartEncoderBoundary.from_pretrained('facebook/bart-base')

Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartEncoderBoundary: ['encoder.layers.0.self_attn_layer_norm.bias', 'encoder.layers.5.self_attn.q_proj.weight', 'encoder.layers.5.self_attn_layer_norm.weight', 'decoder.layers.0.encoder_attn.q_proj.weight', 'encoder.layers.0.final_layer_norm.weight', 'decoder.layers.1.encoder_attn.k_proj.weight', 'decoder.layers.2.encoder_attn.out_proj.bias', 'decoder.layers.3.self_attn.out_proj.bias', 'decoder.layers.3.self_attn_layer_norm.weight', 'decoder.layers.4.final_layer_norm.bias', 'decoder.layers.5.self_attn_layer_norm.bias', 'decoder.layers.0.fc1.weight', 'decoder.layers.2.self_attn_layer_norm.bias', 'decoder.layers.2.self_attn.out_proj.weight', 'encoder.layers.2.self_attn.v_proj.bias', 'decoder.layers.2.fc2.weight', 'decoder.layers.4.self_attn.out_proj.weight', 'decoder.layers.4.encoder_attn_layer_norm.bias', 'decoder.layers.4.final_layer_norm.weight', 'encoder.layers.5.final_layer_norm.weight', 'deco

In [17]:
boundary = torch.LongTensor([(start_position,end_position),]) 

In [18]:
eout = encoder(passage_seq.view(1,-1),passage_attention.view(1,-1), boundaries = boundary)

In [20]:
eout.cleaned_mask.shape

torch.Size([1, 55])